# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
J. Shi  ->  J. Shi  |  ['J. Shi']
J. Liu  ->  J. Liu  |  ['J. Liu']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
Arxiv has 62 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2508.00059


extracting tarball to tmp_2508.00059... done.
Retrieving document from  https://arxiv.org/e-print/2508.00209
extracting tarball to tmp_2508.00209...

 done.


E. Bañados  ->  E. Bañados  |  ['E. Bañados']


...

extracting tarball to tmp_2508.00550... done.
Retrieving document from  https://arxiv.org/e-print/2508.00672


extracting tarball to tmp_2508.00672... done.
Retrieving document from  https://arxiv.org/e-print/2508.00818


extracting tarball to tmp_2508.00818...

 done.


/tmp/ipykernel_3373/2822249172.py:52: LatexWarning: 2508.00818 did not run properly
bad escape \o at position 11
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.00209-b31b1b.svg)](https://arxiv.org/abs/2508.00209) | **Accretion disc reverberation mapping in a high-redshift quasar**  |
|| F. P. Nuñez, <mark>E. Bañados</mark>, S. Panda, J. Heidt |
|*Appeared on*| *2025-08-04*|
|*Comments*| *Accepted for publication in Astronomy & Astrophysics*|
|**Abstract**|            Powered by supermassive black holes at their centers, quasars are among the most luminous objects in the Universe, serving as important probes of cosmic history and galaxy evolution. The size of the accretion disc surrounding the black hole is a critical parameter for understanding quasar physics and their potential use as standard candles in cosmology. However, direct measurements of accretion disc sizes have so far been confined to the Local Universe ($z<0.2$), limiting our understanding of quasars during the peak of cosmic activity. Here, we report the first direct measurement of the accretion disc size in the quasar QSO J0455-4216 at $z=2.66$, when the Universe was only $\sim2$ Gyrs old. Medium-band filters mounted on the MPG/ESO 2.2-metre telescope at La Silla Observatory were used to isolate continuum emission regions during a six-month monitoring campaign. The light curves exhibit pronounced variability features and enabled the detection of inter-band time delays from different parts of the disc. We mapped the disc and located its ultraviolet-emitting outermost region at $3.02^{+0.33}_{-0.57}$ light-days from the black hole ($\sim 500$ AU). Given a supermassive black hole 900 million times the mass of the Sun, these measurements validate accretion disc theory at an unprecedented redshift and pave the way for efficient black hole mass estimates, reducing decades-long spectroscopic reverberation campaigns to just a few years or less.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.00059-b31b1b.svg)](https://arxiv.org/abs/2508.00059) | **The Discovery of 25 um Interstellar Methanol**  |
|| S. L. Nickerson, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-08-04*|
|*Comments*| *Accepted for publication in The Astrophysical Journal Letters (ApJL); 21 pages, 6 Figures*|
|**Abstract**|            We present the first astrophysical detection of methanol (CH3OH) in the torsional band near 25 um. Using high resolution mid-infrared (MIR) spectroscopy, we identified over seventy gas-phase CH3OH absorption lines between 20 and 28 um towards the massive protostar NGC 7538 IRS 1 with SOFIA/EXES. We derive a temperature of 180 K and a total column density of 2 x 10^17 cm-2, comparable to sub-mm measurements. Complementary analysis of acetylene (C2H2) absorption lines is also included. Both CH3OH and C2H2 reveal an unresolved second velocity component. These MIR absorption lines likely probe the molecular material in two edge-on disks, supporting the scenario that NGC 7538 IRS 1 consists of multiple protostars. We provide an updated line list for the torsional band of CH3OH, which was generated from lab work and model calculations. This discovery and the updated line list will enable the search for CH3OH in JWST/MIRI spectra.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.00405-b31b1b.svg)](https://arxiv.org/abs/2508.00405) | **Effect of Matter Accretion on Lithium Enhancement of Giants**  |
|| X. Li, et al. -- incl., <mark>J. Shi</mark> |
|*Appeared on*| *2025-08-04*|
|*Comments*| *Accepted by ApJ*|
|**Abstract**|            A subset of low-mass giants ($<2.2\,M_{\odot}$) exhibit anomalous lithium enhancement behavior, which is still an open topic. Given that more massive giants retain more surface lithium, increasing mass by accreting circumstellar matter could be a channel to enrich lithium. We evaluate this process in the current work. Using MESA, we construct a model of matter accretion, including mass loss, that evolves a star from the main sequence turnoff to the red giant branch tip. The mean accretion rate is estimated from the upper limit of the accreted mass and the evolutionary time of the star during this period, and a grid of accretion rates is constructed. We separately consider their effects on the lithium enhancement of giants, both in terms of the mass and the composition of accretion. Accreting matter with higher lithium abundances has a promoting effect on the lithium enhancement of giants. The accreted matter with excess lithium alleviates the dilution of lithium in the convective envelope during the first dredge-up. The added mass results in lower temperatures at the bottom of the convective envelope, which likewise weakens the depletion of surface lithium. Weak accretion of circumstellar matter is a possible route to lithium enhancement for giants, and it predicts an upper limit on the lithium abundance of $\rm \sim 2.5\,dex$. However, the mass increment it requires poses a potential challenge to real astrophysical environments. Such accretion suppresses lithium dilution and depletion of the star during the first dredge-up, thus exhibiting lithium enhancement behavior.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.00550-b31b1b.svg)](https://arxiv.org/abs/2508.00550) | **A late-time view of the progenitor candidates of the Type II-P SN 2009ib and SN 2012ec**  |
|| Y.-H. Zhao, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-08-04*|
|*Comments*| *Submitted to ApJL. 9 pages, 6 figures*|
|**Abstract**|            The progenitors of Type II-P supernovae (SNe) are generally considered to be red supergiants; however, the so-called "red supergiant problem" indicates that a deeper investigation into the progenitors of this class of SNe is necessary. SN 2009ib and SN 2012ec are two Type II-P SNe for which progenitor candidates have been identified in pre-explosion images. In this work, we use new, late-time Hubble Space Telescope observations to search for the disappearance of these two candidates and confirm their nature. In the case of SN 2009ib, the late-time high-resolution imaging reveals that the progenitor candidate is in fact a blend of multiple unresolved stars. Subsequent difference imaging shows no significant change in brightness at the SN's position even years after the explosion. These findings indicate that the flux from the previously identified source is dominated by unresolved field stars, with little to no contribution from the genuine progenitor. In the case of SN 2012ec, a comparison of pre-explosion and late-time images reveals that the progenitor candidate faded by about 0.6 mag in the F814W band seven years after the explosion, confirming the disappearance of the progenitor.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.00672-b31b1b.svg)](https://arxiv.org/abs/2508.00672) | **On type 1 active galactic nuclei with double-peaked [O~{\sc iii}]. II. properties of long-term optical variability**  |
|| Q. Zheng, X. Zhu, <mark>X. Zhang</mark>, Q. Yuan |
|*Appeared on*| *2025-08-04*|
|*Comments*| *11 pages, 6 figures, accepted to be published in ApJ*|
|**Abstract**|            Double-peaked \oiii~profiles could potentially indicate kiloparsec-scale dual AGNs. We analyze long-term optical light curves of 35 type 1 AGNs with such features from our recent catalog in Zheng et al. (2025). These light curves are obtained from the Catalina Sky Survey and modeled using a Damped Random Walk (DRW) process. A control sample of 210 normal type 1 AGNs matched in redshift, intrinsic luminosity, and black hole mass is also studied. If the double-peaked \oiii~are caused by two type 1 AGNs (dual type 1 AGN), then the combined variability from the two AGNs would be expected to differ from that of a single type 1 AGN. However, there is no statistically significant difference in the variability timescale $\tau$ and intrinsic variability amplitude $\sigma$ between these double-peaked AGNs and the control sample of 210 normal type 1 AGNs. Crucially, computer simulations reveal that dual AGN systems systematically produce lower variability amplitudes than single AGNs, which is inconsistent with the observed variability properties of our double-peaked \oiii~sample. Moreover, simulations suggest that the fraction of dual type 1 AGNs is $\sim$ 3\%, indicating that double-peaked \oiii~may not be a reliable indicator of dual type 1 AGNs in these systems. However, this does not rule out the possibility that some objects may still host dual AGNs involving other combinations, such as type 1+type 2 AGNs. Future studies with larger samples and higher-quality light curves will help clarify the true nature of these systems.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.00818-b31b1b.svg)](https://arxiv.org/abs/2508.00818) | **A Large Catalog of DA White Dwarf Characteristics Using SDSS and Gaia Observations**  |
|| N. R. Crumpler, et al. -- incl., <mark>H.-W. Rix</mark> |
|*Appeared on*| *2025-08-04*|
|*Comments*| **|
|**Abstract**|            We present a catalog of 8545 and 19,257 unique DA white dwarfs observed in SDSS Data Release 19 and previous SDSS data releases, respectively. This is the largest catalog of both spectroscopic and photometric measurements of DA white dwarfs available to date, and we make this catalog and all code used to create it publicly available. We measure the apparent radial velocity, spectroscopic effective temperature and surface gravity, and photometric effective temperature and radius for all objects in our catalog. We validate our measurements against other published white dwarf catalogs. For apparent radial velocities, surface gravities, and effective temperatures measured from spectra with signal-to-noise ratios $>50$, our measurements agree with published SDSS white dwarf catalogs to within 7.5 km/s, 0.060 dex, and $2.4\%$, respectively. For radii and effective temperatures measured with Gaia photometry, our measurements agree with other published Gaia datasets to within $0.0005$ $R_\odot$ and $3\%$, respectively. We use this catalog to investigate systematic discrepancies between white dwarfs observed in SDSS-V and previous generations of SDSS. For objects observed in both SDSS-V and previous generations, we uncover systematic differences between measured spectroscopic parameters depending on which set of survey data is used. On average, the measured apparent radial velocity of a DA white dwarf is $11.5$ km/s larger and the surface gravity is $0.015$ dex smaller when a white dwarf's spectroscopic parameters are measured using SDSS-V data compared to using data from previous generations of SDSS. These differences may be due to changes in the wavelength solution across survey generations.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \o at position 11</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2508.00209/./figures/lcs_ap.png', 'tmp_2508.00209/./figures/GPCC_delay_wave.png', 'tmp_2508.00209/./figures/AD_TF_models.png']
copying  tmp_2508.00209/./figures/lcs_ap.png to _build/html/
copying  tmp_2508.00209/./figures/GPCC_delay_wave.png to _build/html/
copying  tmp_2508.00209/./figures/AD_TF_models.png to _build/html/
exported in  _build/html/2508.00209.md
    + _build/html/tmp_2508.00209/./figures/lcs_ap.png
    + _build/html/tmp_2508.00209/./figures/GPCC_delay_wave.png
    + _build/html/tmp_2508.00209/./figures/AD_TF_models.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\bd}[1]{\mbox{\boldmath #1}}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\civ}{{C\sc{iv}}\/}$
$\newcommand{\ciii}{{C\sc{iii}]}\/}$</div>



<div id="title">

# Accretion disc reverberation mapping in a high-redshift quasar

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.00209-b31b1b.svg)](https://arxiv.org/abs/2508.00209)<mark>Appeared on: 2025-08-04</mark> -  _Accepted for publication in Astronomy & Astrophysics_

</div>
<div id="authors">

F. P. Nuñez, <mark>E. Bañados</mark>, S. Panda, J. Heidt

</div>
<div id="abstract">

**Abstract:** Powered by supermassive black holes at their centers, quasars are among the most luminous objects in the Universe, serving as important probes of cosmic history and galaxy evolution. The size of the accretion disc surrounding the black hole is a critical parameter for understanding quasar physics and their potential use as standard candles in cosmology. However, direct measurements of accretion disc sizes have so far been confined to the Local Universe ( $z<0.2$ ), limiting our understanding of quasars during the peak of cosmic activity. Here, we report the first direct measurement of the accretion disc size in the quasar QSO J0455-4216 at $z=2.66$ , when the Universe was only $\sim2$ Gyrs old. Medium-band filters mounted on the MPG/ESO 2.2-metre telescope at La Silla Observatory were used to isolate continuum emission regions during a six-month monitoring campaign. The light curves exhibit pronounced variability features and enabled the detection of inter-band time delays from different parts of the disc. We mapped the disc and located its ultraviolet-emitting outermost region at $\( 3.02^{+0.33}_{-0.57} \)$ light-days from the black hole ( $\( \sim 500 \)$ AU). Given a supermassive black hole 900 million times the mass of the Sun, these measurements validate accretion disc theory at an unprecedented redshift and pave the way for efficient black hole mass estimates, reducing decades-long spectroscopic reverberation campaigns to just a few years or less.

</div>

<div id="div_fig1">

<img src="tmp_2508.00209/./figures/lcs_ap.png" alt="Fig1" width="100%"/>

**Figure 1. -** Multi-band light curves of QSO J0455-4216. Variability of around 7\% is observed across all wavelengths for the entire campaign (6 months), while a smaller variability of about 5\% is present within individual months of observations. Micro-variability on the order of 2\%-3\% is evident on timescales of days, with delayed features between bands of a few days. For comparison, the light curve of a representative reference star in the field used in constructing the AGN light curves is shown at the bottom. Dotted lines indicate a 1\% variability and all light curves are vertically shifted for clarity. (*fig:Figure1*)

</div>
<div id="div_fig2">

<img src="tmp_2508.00209/./figures/GPCC_delay_wave.png" alt="Fig2" width="100%"/>

**Figure 2. -** Observed time delay spectrum. The shaded regions represent the time delay ($\tau$) probability distributions obtained using the GPCC method (\citealt{2023A&A...674A..83P}), with black dots indicating the most likely delay corresponding to the peak of each distribution. The solid line denotes the expected delay from an optically thick, geometrically thin accretion disc irradiated by a lamp post geometry. A summary of the physical parameters used in the model, including the black hole mass ($M_{\rm BH}$), luminosity ($\lambda L_{\lambda_{1350}}$), and redshift ($z$), is provided. (*fig:Figure2*)

</div>
<div id="div_fig3">

<img src="tmp_2508.00209/./figures/AD_TF_models.png" alt="Fig3" width="100%"/>

**Figure 3. -** _Left_: Normalized accretion disc transfer functions \(\Psi(\tau|\lambda)\) for the four photometric bands: \( 4858   \text{Å} \)(blue), \( 5315   \text{Å} \)(green), \( 6043   \text{Å} \)(orange), and \( 7705   \text{Å} \)(red). As predicted by standard thin-disc (SS73) and lamp-post models, the transfer functions broaden with increasing wavelength. Longer-wavelength emission arises from larger radii, resulting in longer time delays due to the greater light travel distance. _Right_: Observed normalized continuum light curves (points with error bars) compared with model predictions. Dashed lines show the results from a single-band modelling approach, in which a latent signal is inferred independently for each filter. Solid black lines show the model light curves computed from the joint GPCC-inferred latent signal, convolved with the corresponding transfer functions. The shaded regions represent the \( 1\sigma \) uncertainties propagated through the model scaling process; see Appendix \ref{ad:simul} for details. (*fig:Figure3*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.00209"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

133  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
